In [ ]:
import random
import numpy as np 
import pickle
import json
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Activation , Dropout
from tensorflow.keras.optimizers import SGD
import tensorflow as tf
from google.colab import drive
import pandas as pd



In [ ]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
lemmatizer = WordNetLemmatizer()


In [ ]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
data = pd.read_json('applai_dataa.json')

In [ ]:
data['applai'].keys

<bound method Series.keys of 0    {'tag': 'greetings', 'patterns': ['hello', 'hi...
1    {'tag': 'good bye', 'patterns': ['bye', 'bye b...
2    {'tag': 'name', 'patterns': ['what is your nam...
3    {'tag': 'applai`s trainings time', 'patterns':...
4    {'tag': 'applai`s trainings place', 'patterns'...
5    {'tag': 'applai`s departments', 'patterns': ['...
6    {'tag': 'applai`s trainings place', 'patterns'...
7    {'tag': 'applai`s campaigns', 'patterns': ['Do...
8    {'tag': 'general information about applai', 'p...
9    {'tag': 'applai's training content', 'patterns...
Name: applai, dtype: object>

In [ ]:
words=[]
documents=[]
classes=[]
Ignore_letters=['?','!','.',',']

In [ ]:
for i in data['applai']:
  for pattern in i['patterns']:
    word_list = nltk.word_tokenize(pattern)
#word_tokenize using for separating each word in the sentence in each pattern
    words.extend(word_list)
    documents.append((word_list, i['tag']))
#so we could know that this word in the pattern belong to this tag
    if i['tag'] not in classes :
      classes.append(i['tag'])       

In [ ]:
print(documents[0])

(['hello'], 'greetings')


In [ ]:
from nltk.stem import WordNetLemmatizer

#wnl = WordNetLemmatizer()

lemmatized_words = [[lemmatizer.lemmatize(word) for word in l if word not in Ignore_letters] for l in words ]
words = sorted(set(words))
classes = sorted(set(classes))



In [ ]:
print(words)

["'s", '?', 'AI', 'Ain', 'Can', 'Does', 'For', 'How', 'I', 'Is', 'Shams', 'University', 'What', 'When', '`', 'activities', 'and', 'any', 'applai', 'are', 'at', 'been', 'branches', 'bye', 'call', 'campaigns', 'club', 'content', 'could', 'cover', 'created', 'day', 'deep', 'departments', 'does', 'established', 'explain', 'find', 'go', 'good', 'greetings', 'has', 'have', 'held', 'hello', 'hey', 'hi', 'hold', 'how', 'in', 'is', 'learning', 'leaving', 'm', 'machine', 'many', 'me', 'name', 'now', 'offer', 'offered', 'other', 'place', 'related', 's', 'should', 'tell', 'the', 'there', 'topic', 'training', 'trainings', 'up', 'was', 'what', 'when', 'where', 'who', 'will', 'working', 'years', 'you', 'your']


In [ ]:
print(classes)

["applai's training content", 'applai`s campaigns', 'applai`s departments', 'applai`s trainings place', 'applai`s trainings time', 'general information about applai', 'good bye', 'greetings', 'name']


In [ ]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))
#to save the lists

In [ ]:
training=[]
output_empty=[0] * len(classes)

In [ ]:
for document in documents:
  bag=[]
  word_patterns=document[0]
  word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)

  output_row = list(output_empty)
  output_row[classes.index(document[1])] = 1
  training.append([bag , output_row])  

In [ ]:
random.shuffle(training)
training = np.array(training)

<ipython-input-198-cc0bc1dae1db>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [ ]:
train_x = list(training[:,0])
train_y = list(training[:,1])

In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='Softmax'))
sgd = tf.keras.optimizers.legacy.SGD(
    learning_rate=0.01,decay=1e-6, 
    momentum=0.9,
    nesterov=False,
    name='SGD'
)
model.compile(loss='categorical_crossentropy', optimizer = sgd , metrics=['accuracy'])

hist=model.fit(np.array(train_x),np.array(train_y),epochs = 100 , batch_size = 5 , verbose = 1 )
model.save('applai_chatbot_model.h5',hist)


Epoch 1/100
8/8 [==============================] - 0s 2ms/step - loss: 2.2763 - accuracy: 0.0256
Epoch 2/100
8/8 [==============================] - 0s 2ms/step - loss: 2.1835 - accuracy: 0.1282
Epoch 3/100
8/8 [==============================] - 0s 2ms/step - loss: 2.0407 - accuracy: 0.3846
Epoch 4/100
8/8 [==============================] - 0s 2ms/step - loss: 1.8664 - accuracy: 0.4359
Epoch 5/100
8/8 [==============================] - 0s 2ms/step - loss: 2.0041 - accuracy: 0.3333
Epoch 6/100
8/8 [==============================] - 0s 2ms/step - loss: 1.8502 - accuracy: 0.3590
Epoch 7/100
8/8 [==============================] - 0s 2ms/step - loss: 1.7155 - accuracy: 0.4872
Epoch 8/100
8/8 [==============================] - 0s 2ms/step - loss: 1.5577 - accuracy: 0.5385
Epoch 9/100
8/8 [==============================] - 0s 2ms/step - loss: 1.4781 - accuracy: 0.6923
Epoch 10/100
8/8 [==============================] - 0s 2ms/step - loss: 1.4706 - accuracy: 0.6154
Epoch 11/100
8/8 [===========

In [ ]:
!pip install streamlit-chat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# #%%writefile app.py
# import random
# import json
# import pickle
# import numpy as np
# import nltk
# from nltk.stem import WordNetLemmatizer
# from tensorflow.keras.models import load_model
# import streamlit as st
# import tensorflow as tf
# from keras.models import load_model
# import nltk
# # %%writefile app.py
# import streamlit as st
# from streamlit_chat import message

# # message("My message") 
# # message("Hello bot!", is_user=True)  # align's the message to the right

# from PIL import Image



# lemmatizer = WordNetLemmatizer()
# intents = json.loads(open('applai_dataa.json').read())
# words = pickle.load(open('words.pkl', 'rb'))
# classes = pickle.load(open('classes.pkl', 'rb'))
# model = load_model('applai_chatbot_model.h5')
# def clean_up_sentence (sentence):
#   sentence_words=nltk.word_tokenize (sentence)
#   sentence_words= [lemmatizer.lemmatize (word) for word in sentence_words]
#   return sentence_words
# def bag_of_words (sentence):
#   sentence_words = clean_up_sentence (sentence)
#   bag = [0] * len(words)
#   for w in sentence_words:
#     for i, word in enumerate(words):
#       if word == w:
#         bag[i] = 1
#     return np.array(bag)  
# def predict_class(sentence):
#   bow = bag_of_words (sentence)
#   res = model.predict(np.array([bow]))[0]
#   ERROR_THRESHOLD = 0.25
#   results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
#   results.sort(key=lambda x: x[1], reverse=True)
#   return_list = []
#   for r in results:
#     return_list.append({'applai': classes [r[0]], 'probability': str(r[1])})
#   return return_list
# model = tf.keras.models.load_model('applai_chatbot_model.h5')
# nltk.download('punkt')


# def get_response(intents_list, intents_json):
#   tag = intents_list[0]['applai'] 
#   list_of_intents = intents_json['applai']
#   for i in list_of_intents:
#     if i['tag'] == tag:
#       result = random.choice(i['responses'])
#       break
#   return result
# def generate_response(user_input):
#     # Preprocess user input
#     user_input = user_input.lower()
#     tokens = nltk.word_tokenize(user_input)

#     # Convert tokens to a sequence of word indices
#     word_indices = []
#     for word in tokens:
#         if word in word_index:
#             word_indices.append(word_index[word])
#         else:
#             word_indices.append(word_index['<unk>'])

#     # Pad sequence to max length
#     padded_seq = pad_sequences([word_indices], maxlen=max_seq_length,
#                                padding='post', truncating='post')

#     # Generate response
#     response_idx = np.argmax(model.predict(padded_seq), axis=-1)[0]
#     response = index_word[response_idx]

#     return response
# def resp (user_input):
#   x=predict_class(user_input)
#   response = get_response(x,intents)
#   st.write('ApplAI Bot: ',response)



# i=0
# import uuid


# st.title('Welcome To Applai Chatbot')
# st.sidebar.image("ApplAiOnly_Logo.png", use_column_width=True)
# conversation_active=True



# # sub=False
# # user_input=" "
# # flag=True
# # while user_input!='Exit' :
# #   if flag:
# #     user_input = st.text_input('Enter a question:')
# #     sub=st.button('Get answer')
# #     #flag=False
 
# #   if sub :
# #     flag=True
# #     sub=False
# #     x=predict_class(user_input)
# #     response = get_response(x,intents)
# #     st.write(response)
# once = True
# sub = False

# if once:
    
#     user_input = st.text_input('Enter a question:')
#     sub = st.button('Get answer')

# if sub:
#     once = False
#     sub = not sub
#     x = predict_class(user_input)
#     response = get_response(x, intents)
#     st.write(response)
# if not once:
#   user_input = st.text_input('Enter another question:')
#   sub = st.button('Get answer',str(uuid.uuid4()))

#     #user_input2 = st.text_input('Enter another question:')
#     #sub2=st.button('Get answer',key=2)



      









In [ ]:
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model
import streamlit as st
import tensorflow as tf
from keras.models import load_model
import nltk
# %%writefile app.py
import streamlit as st
from streamlit_chat import message

# message("My message") 
# message("Hello bot!", is_user=True)  # align's the message to the right

from PIL import Image



lemmatizer = WordNetLemmatizer()
intents = json.loads(open('applai_dataa.json').read())
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('applai_chatbot_model.h5')
def clean_up_sentence (sentence):
  sentence_words=nltk.word_tokenize (sentence)
  sentence_words= [lemmatizer.lemmatize (word) for word in sentence_words]
  return sentence_words
def bag_of_words (sentence):
  sentence_words = clean_up_sentence (sentence)
  bag = [0] * len(words)
  for w in sentence_words:
    for i, word in enumerate(words):
      if word == w:
        bag[i] = 1
    return np.array(bag)  
def predict_class(sentence):
  bow = bag_of_words (sentence)
  res = model.predict(np.array([bow]))[0]
  ERROR_THRESHOLD = 0.25
  results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in results:
    return_list.append({'applai': classes [r[0]], 'probability': str(r[1])})
  return return_list
model = tf.keras.models.load_model('applai_chatbot_model.h5')
nltk.download('punkt')


def get_response(intents_list, intents_json):
  tag = intents_list[0]['applai'] 
  list_of_intents = intents_json['applai']
  for i in list_of_intents:
    if i['tag'] == tag:
      result = random.choice(i['responses'])
      break
  return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import gradio as gr
def chatbot(input, history=[]):
    x = predict_class(input)
    response = get_response(x, intents)
  
    #output = openai_chat(input)
    history.append((input, response))
    return history, history
add_image = "body {background-image: 'ApplAiOnly_Logo.png';}"
#with gr.Blocks(css=".gradio-container {background-image: url('file=clouds.jpg')}") as demo:
#css=".gradio-container {background-image: url('file=ApplAiOnly_Logo.png')}"
iface=gr.Interface(fn = chatbot,logo='ApplAiOnly_Logo.png',title="Welcome To ApplAI ChatBot"
,
             inputs = ["text",'state'],
             outputs = ["chatbot",'state'])


iface.launch()



AttributeError: ignored

In [ ]:
print(gr.__version__)



3.19.1


In [ ]:
# import streamlit as st
# from streamlit_chat import message as st_message
# if "history" not in st.session_state:
#     st.session_state.history = []

# st.title('Welcome To Applai Chatbot')
# st.sidebar.image("ApplAiOnly_Logo.png", use_column_width=True)


# def generate_answer():
#     tokenizer, model = get_models()
#     user_message = st.session_state.input_text
#     inputs = tokenizer(st.session_state.input_text, return_tensors="pt")
#     result = model.generate(**inputs)
#     message_bot = tokenizer.decode(
#         result[0], skip_special_tokens=True
#     )  # .replace("<s>", "").replace("</s>", "")

#     st.session_state.history.append({"message": user_message, "is_user": True})
#     st.session_state.history.append({"message": message_bot, "is_user": False})


# st.text_input("Talk to the bot", key="input_text", on_change=generate_answer)

# for chat in st.session_state.history:
#     st_message(**chat)  # unpacking

In [ ]:
# #%%writefile app.py

# import streamlit as st
# import random
# import json
# import pickle
# import numpy as np
# import nltk
# from nltk.stem import WordNetLemmatizer
# from tensorflow.keras.models import load_model
# import streamlit as st
# import tensorflow as tf
# from keras.models import load_model
# import nltk
# import uuid

# from PIL import Image



# lemmatizer = WordNetLemmatizer()
# intents = json.loads(open('applai_dataa.json').read())
# words = pickle.load(open('words.pkl', 'rb'))
# classes = pickle.load(open('classes.pkl', 'rb'))
# model = load_model('applai_chatbot_model.h5')
# def clean_up_sentence (sentence):
#   sentence_words=nltk.word_tokenize (sentence)
#   sentence_words= [lemmatizer.lemmatize (word) for word in sentence_words]
#   return sentence_words
# def bag_of_words (sentence):
#   sentence_words = clean_up_sentence (sentence)
#   bag = [0] * len(words)
#   for w in sentence_words:
#     for i, word in enumerate(words):
#       if word == w:
#         bag[i] = 1
#     return np.array(bag)  
# def predict_class(sentence):
#   bow = bag_of_words (sentence)
#   res = model.predict(np.array([bow]))[0]
#   ERROR_THRESHOLD = 0.25
#   results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
#   results.sort(key=lambda x: x[1], reverse=True)
#   return_list = []
#   for r in results:
#     return_list.append({'applai': classes [r[0]], 'probability': str(r[1])})
#   return return_list
# model = tf.keras.models.load_model('applai_chatbot_model.h5')
# nltk.download('punkt')


# def get_response(intents_list, intents_json):
#   tag = intents_list[0]['applai'] 
#   list_of_intents = intents_json['applai']
#   for i in list_of_intents:
#     if i['tag'] == tag:
#       result = random.choice(i['responses'])
#       break
#   return result
# def generate_response(user_input):
#     # Preprocess user input
#     user_input = user_input.lower()
#     tokens = nltk.word_tokenize(user_input)

#     # Convert tokens to a sequence of word indices
#     word_indices = []
#     for word in tokens:
#         if word in word_index:
#             word_indices.append(word_index[word])
#         else:
#             word_indices.append(word_index['<unk>'])

#     # Pad sequence to max length
#     padded_seq = pad_sequences([word_indices], maxlen=max_seq_length,
#                                padding='post', truncating='post')

#     # Generate response
#     response_idx = np.argmax(model.predict(padded_seq), axis=-1)[0]
#     response = index_word[response_idx]

#     return response

# # Set page title
# ####################################################################
# st.set_page_config(page_title="'Welcome To Applai Chatbot'")

# # Define a function to handle user input and bot responses
# def run_chatbot():
#     chat_history = []
    
#     while True:
#         key1 = str(uuid.uuid4())
#         key2 = str(uuid.uuid4())
#         key3 = str(uuid.uuid4())
#         key4 = str(uuid.uuid4())
#         # Create an input field for the user to enter their question
#         user_input = st.text_input("Enter your question here:",key=key1)
        
#         # Create a button to submit the user's question
#         if st.button("Ask",key=key2):
#             if user_input:
#                 # Add the user's question to the chat history
#                 chat_history.append("You: " + user_input)
                
#                 # Call the chatbot function to generate a response
#                 x=predict_class(user_input)
#                 response = get_response(x,intents)
                
                
                
#                 # Add the bot's response to the chat history
#                 chat_history.append("Bot: " + response)
            
#         # Display the chat history
#         #st.write("\n".join(chat_history))

# # Run the chatbot function
# run_chatbot()






In [ ]:
# import streamlit as st

# # Define a function to generate the chatbot response
# def generate_response(input_text):
#     # Add your chatbot logic here
#     return "You said: " + input_text

# # Loop to repeatedly prompt the user for input and generate a response
# conversation_active = True
# while conversation_active:
#     # Create a text box for user input
#     user_input = st.text_input("Enter your question or message here:")

#     # Create a button that the user can click to submit their input
#     submit_button = st.button("Submit")

#     # If the user has submitted input, generate and display the chatbot response
#     if submit_button:
#         response = generate_response(user_input)
#         st.write(response)

#         # Prompt the user for another question
#         user_input = st.text_input("Enter another question or message, or leave blank to end the conversation:")
#         if user_input == "":
#             conversation_active = False

In [ ]:

# #%%writefile app.py
# import random
# import json
# import pickle
# import numpy as np
# import nltk
# from nltk.stem import WordNetLemmatizer
# from tensorflow.keras.models import load_model
# import streamlit as st
# import tensorflow as tf
# from keras.models import load_model
# import nltk

# from PIL import Image



# lemmatizer = WordNetLemmatizer()
# intents = json.loads(open('applai_dataa.json').read())
# words = pickle.load(open('words.pkl', 'rb'))
# classes = pickle.load(open('classes.pkl', 'rb'))
# model = load_model('applai_chatbot_model.h5')
# def clean_up_sentence (sentence):
#   sentence_words=nltk.word_tokenize (sentence)
#   sentence_words= [lemmatizer.lemmatize (word) for word in sentence_words]
#   return sentence_words
# def bag_of_words (sentence):
#   sentence_words = clean_up_sentence (sentence)
#   bag = [0] * len(words)
#   for w in sentence_words:
#     for i, word in enumerate(words):
#       if word == w:
#         bag[i] = 1
#     return np.array(bag)  
# def predict_class(sentence):
#   bow = bag_of_words (sentence)
#   res = model.predict(np.array([bow]))[0]
#   ERROR_THRESHOLD = 0.25
#   results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
#   results.sort(key=lambda x: x[1], reverse=True)
#   return_list = []
#   for r in results:
#     return_list.append({'applai': classes [r[0]], 'probability': str(r[1])})
#   return return_list
# model = tf.keras.models.load_model('applai_chatbot_model.h5')
# nltk.download('punkt')


# def get_response(intents_list, intents_json):
#   tag = intents_list[0]['applai'] 
#   list_of_intents = intents_json['applai']
#   for i in list_of_intents:
#     if i['tag'] == tag:
#       result = random.choice(i['responses'])
#       break
#   return result
# def generate_response(user_input):
#     # Preprocess user input
#     user_input = user_input.lower()
#     tokens = nltk.word_tokenize(user_input)

#     # Convert tokens to a sequence of word indices
#     word_indices = []
#     for word in tokens:
#         if word in word_index:
#             word_indices.append(word_index[word])
#         else:
#             word_indices.append(word_index['<unk>'])

#     # Pad sequence to max length
#     padded_seq = pad_sequences([word_indices], maxlen=max_seq_length,
#                                padding='post', truncating='post')

#     # Generate response
#     response_idx = np.argmax(model.predict(padded_seq), axis=-1)[0]
#     response = index_word[response_idx]

#     return response

# def main():
#     i=0
#     st.title('Welcome To Applai Chatbot')
#     st.sidebar.image("ApplAiOnly_Logo.png", use_column_width=True)
#     while True:
#       if(i==0):
#         user_input = st.text_input('Enter a question:',key=i)
#       else:
#         user_input = st.text_input('Enter a question:',key=i)
#       if st.button('Get answer',key=i):
#           x=predict_class(user_input)
#           response = get_response(x,intents)
#           st.write(response)
#       i+=1
      




# if __name__ == '__main__':
#     # main()

In [ ]:
#%%writefile app.py


# def generate_response(user_input):
#     # Preprocess user input
#     user_input = user_input.lower()
#     tokens = nltk.word_tokenize(user_input)

#     # Convert tokens to a sequence of word indices
#     word_indices = []
#     for word in tokens:
#         if word in word_index:
#             word_indices.append(word_index[word])
#         else:
#             word_indices.append(word_index['<unk>'])

#     # Pad sequence to max length
#     padded_seq = pad_sequences([word_indices], maxlen=max_seq_length,
#                                padding='post', truncating='post')

#     # Generate response
#     response_idx = np.argmax(model.predict(padded_seq), axis=-1)[0]
#     response = index_word[response_idx]

#     return response

# def main():
#     st.title('Chatbot')
#     user_input = st.text_input('Enter a question:')
#     if st.button('Get answer'):
#         response = generate_response(user_input)
#         st.write(response)

# if __name__ == '__main__':
#     main()


In [ ]:
# import streamlit as st
# import tensorflow as tf

# # Load your chatbot model
# model = tf.keras.models.load_model('applai_chatbot_model.model')

# # Create a Streamlit app
# def app():
#     # Create a text input field
#     user_input = st.text_input('You:', '')

#     # Define the chatbot function
#     def chatbot_response(user_input):
#         # Preprocess the user's input
#         # and generate the chatbot's response
#         # using your loaded model
#         response = "Chatbot: " + "Your response here"

#         return response

#     # Use the Streamlit display functions
#     if user_input:
#         response = chatbot_response(user_input)
#         st.write(response)

# # Run the app
# if __name__ == '__main__':
#     app()


In [ ]:
#!pip install pyngrok

In [ ]:
# %%writefile app.py
# import streamlit as st
# from streamlit_chat import message

# message("My message") 
# message("Hello bot!", is_user=True)  # align's the message to the right

In [ ]:
!streamlit run app.py&>/dev/null&

In [ ]:
!ngrok authtoken 2Lyi5BxnZ4GkA1BwlJtM2IWkWHj_7aaWYYnBWL58p5xWNLRq2

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
#!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

In [ ]:
#!unzip ngrok-stable-linux-amd64.zip

In [ ]:
get_ipython().system_raw('./ngrok http 8501 &')


In [ ]:
import time
time.sleep(2)

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://fd5f-34-173-162-33.ngrok.io


In [ ]:
!streamlit run /content/app.py




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8502
  External URL: http://34.173.162.33:8502

  Stopping...
^C


In [ ]:
!pip install streamlit-chat 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#######################

In [ ]:
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 93.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.7/140.7 KB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import gradio as gr


In [ ]:
def chatbot(input, history=[]):
    output = openai_chat(input)
    history.append((input, output))
    return history, history

In [ ]:
gr.Interface(fn = chatbot,
             inputs = ["text",'state'],
             outputs = ["chatbot",'state']).launch(debug = True)